In [1]:
import os


In [3]:
%pwd

'c:\\Chandu\\WorkSpace\\Learnings\\MachineLearning\\Text-Summarizer-Project\\research'

In [8]:
os.chdir("../")

In [9]:
%pwd

'c:\\Chandu\\WorkSpace\\Learnings\\MachineLearning\\Text-Summarizer-Project'

In [10]:
#entity is to define the return type of the function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(
            self,
            config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
            url = self.config.source_url,
            filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extact_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    
    

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extact_zip_file()
except Exception as e:
    raise e

[2025-03-20 18:37:12,283: INFO: common: yamal file: config\config.yaml loaded succesfully]
[2025-03-20 18:37:12,292: INFO: common: yamal file: params.yaml loaded succesfully]
[2025-03-20 18:37:12,299: INFO: common: created directory at: artifacts]
[2025-03-20 18:37:12,307: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-20 18:37:16,166: INFO: 1878340992: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AA83:3ECF11:ED1BC:2204B4:67DC0C45
Accept-Ranges: bytes
Date: Thu, 20 Mar 2025 13:05:46 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100024-HYD
X-Cache: 